# Equations of motion for the cat-bot

Do imports.

In [81]:
import sympy as sym
import numpy as np
from IPython.display import display, Markdown
from sympy.physics import mechanics
mechanics.init_vprinting()

Define variables and constants as symbols.

In [82]:
# Time
t = sym.Symbol('t')

# Horizontal position of wheel axle
zeta = mechanics.dynamicsymbols('zeta', real=True)

# Angle of body from vertical (positive means leaning forward)
theta = mechanics.dynamicsymbols('theta', real=True)

# Torque applied by the body on the wheel
tau = sym.symbols('tau', real=True)

# Wheel parameters
#   m_w  mass
#   J_w  moment of inertia
#   r_w  radius
m_w, J_w, r_w = sym.symbols('m_w, J_w, r_w', real=True, positive=True)

# Body parameters
#   m_b  mass
#   J_b  moment of inertia (about center-of-mass)
#   r_b  distance between wheel axle and body center-of-mass
m_b, J_b, r_b = sym.symbols('m_b, J_b, r_b', real=True, positive=True)

# Acceleration of gravity
g = sym.symbols('g', real=True, positive=True)

Compute Lagrangian.

In [83]:
# Position of wheel and body
p_w = sym.Matrix([zeta, r_w])
p_b = sym.Matrix([zeta + r_b * sym.sin(theta), r_w + r_b * sym.cos(theta)])

# Linear velocity of wheel and body
v_w = p_w.diff(t)
v_b = p_b.diff(t)

# Angular velocity of wheel (assume rolling without slipping on flat ground) and body
omega_w = zeta.diff(t) / r_w
omega_b = theta.diff(t)

# Kinetic and potential energy
T = (m_w * v_w.dot(v_w) + m_b * v_b.dot(v_b) + J_w * omega_w**2 + J_b * omega_b**2) / 2
V = (m_w * p_w[1] * g) + (m_b * p_b[1] * g)

# Lagrangian
L = sym.Matrix([sym.simplify(T - V)])

Compute and show the matrices $M(q)$, $N(q, \dot{q})$, and $F(q)$ for which the equations of motion can be expressed as

$$ M(q) \ddot{q} + N(q, \dot{q}) = F(q) r $$

where

$$ q = \begin{bmatrix} \zeta \\ \theta \end{bmatrix} \qquad\text{and}\qquad r = \begin{bmatrix} \tau \end{bmatrix}. $$

In [84]:
# Generalized coordinates
q = sym.Matrix([zeta, theta])

# Generalized velocities
v = q.diff(t)

# Coefficients in the equations of motion
M = sym.simplify(L.jacobian(v).jacobian(v))
N = sym.simplify(L.jacobian(v).jacobian(q) @ v - L.jacobian(q).T)
F = sym.simplify(sym.Matrix([(zeta / r_w) - theta]).jacobian(q).T)

# Show results
display(Markdown(f'$$ M(q) = {mechanics.mlatex(M)} $$'))
display(Markdown(f'$$ N(q, \\dot{{q}}) = {mechanics.mlatex(N)} $$'))
display(Markdown(f'$$ F(q) = {mechanics.mlatex(F)} $$'))

$$ M(q) = \left[\begin{matrix}\frac{J_{w}}{r_{w}^{2}} + m_{b} + m_{w} & m_{b} r_{b} \cos{\left(\theta \right)}\\m_{b} r_{b} \cos{\left(\theta \right)} & J_{b} + m_{b} r_{b}^{2}\end{matrix}\right] $$

$$ N(q, \dot{q}) = \left[\begin{matrix}- m_{b} r_{b} \sin{\left(\theta \right)} \dot{\theta}^{2}\\- g m_{b} r_{b} \sin{\left(\theta \right)}\end{matrix}\right] $$

$$ F(q) = \left[\begin{matrix}\frac{1}{r_{w}}\\-1\end{matrix}\right] $$

Define and show the numerical value of constants (consistant with the URDF file).

In [85]:
##########################################
# Temporary variables that can be ignored

# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

##########################################
# Parameters

# Define them
params = {
    r_w: r,
    m_w: 2 * mw,
    J_w: 2 * Jw,
    r_b: h,
    m_b: mb,
    J_b: Jby,
    g: 9.81,
}

# Show them
s = ''
for key, val in params.items():
    s += fr'{key} &= {mechanics.mlatex(val)} \\ '
s = s[:-3]
display(Markdown(fr'$$ \begin{{align*}}{s}\end{{align*}} $$'))

$$ \begin{align*}r_w &= 0.325 \\ m_w &= 2.4 \\ J_w &= 0.12675 \\ r_b &= 0.3 \\ m_b &= 12.0 \\ J_b &= 0.8 \\ g &= 9.81 \end{align*} $$

In [86]:
# Define generalized coordinates and their derivatives
q = sym.Matrix([zeta, theta])
dq = q.diff(t)
ddq = dq.diff(t)

# Define the state vector
m = sym.Matrix([zeta, theta, dq[0], dq[1]])

# Define input
r = sym.Matrix([tau])

# Express the second-order system as first-order
M_inv = M.inv()
ddq_expr = M_inv * (F * r - N)

# Define the first-order ODE system
f_m = sym.Matrix([dq[0], dq[1], ddq_expr[0], ddq_expr[1]])

# Find equilibrium points
m_eq = sym.Matrix([sym.Symbol('zeta_e'), sym.Symbol('theta_e'), 0, 0])
r_eq = sym.Matrix([sym.Symbol('tau_e')])
eqn = f_m.subs({zeta: m_eq[0], theta: m_eq[1], dq[0]: 0, dq[1]: 0, tau: r_eq[0]})
eq_point = sym.solve(eqn, [m_eq[0], m_eq[1], r_eq[0]])


In [87]:
# Define state and input errors
x = m - m_eq  # State deviation from equilibrium
u = r - r_eq  # Input deviation from equilibrium

In [88]:
# Compute A and B matrices
#A = f_m.jacobian(m).subs({zeta: m_eq[0], theta: m_eq[1], dq[0]: 0, dq[1]: 0, tau: r_eq[0]})
#B = f_m.jacobian(r).subs({zeta: m_eq[0], theta: m_eq[1], dq[0]: 0, dq[1]: 0, tau: r_eq[0]})

A = f_m.jacobian(m).subs({zeta: m_eq[0], theta: 0, dq[0]: 0, dq[1]: 0, tau: 0})
B = f_m.jacobian(r).subs({zeta: m_eq[0], theta: 0, dq[0]: 0, dq[1]: 0, tau: 0})

# Display A and B matrices
display(Markdown(f'$$ {mechanics.mlatex(A)} $$'))
display(Markdown(f'$$ {mechanics.mlatex(B)} $$'))

$$ \left[\begin{matrix}0 & 0 & 1 & 0\\0 & 0 & 0 & 1\\0 & - \frac{g m_{b}^{2} r_{b}^{2} r_{w}^{2}}{J_{b} J_{w} + J_{b} m_{b} r_{w}^{2} + J_{b} m_{w} r_{w}^{2} + J_{w} m_{b} r_{b}^{2} + m_{b} m_{w} r_{b}^{2} r_{w}^{2}} & 0 & 0\\0 & \frac{g m_{b} r_{b} \left(J_{w} + m_{b} r_{w}^{2} + m_{w} r_{w}^{2}\right)}{J_{b} J_{w} + J_{b} m_{b} r_{w}^{2} + J_{b} m_{w} r_{w}^{2} + J_{w} m_{b} r_{b}^{2} + m_{b} m_{w} r_{b}^{2} r_{w}^{2}} & 0 & 0\end{matrix}\right] $$

$$ \left[\begin{matrix}0\\0\\\frac{m_{b} r_{b} r_{w}^{2}}{J_{b} J_{w} + J_{b} m_{b} r_{w}^{2} + J_{b} m_{w} r_{w}^{2} + J_{w} m_{b} r_{b}^{2} + m_{b} m_{w} r_{b}^{2} r_{w}^{2}} + \frac{J_{b} r_{w}^{2} + m_{b} r_{b}^{2} r_{w}^{2}}{r_{w} \left(J_{b} J_{w} + J_{b} m_{b} r_{w}^{2} + J_{b} m_{w} r_{w}^{2} + J_{w} m_{b} r_{b}^{2} + m_{b} m_{w} r_{b}^{2} r_{w}^{2}\right)}\\- \frac{m_{b} r_{b} r_{w}}{J_{b} J_{w} + J_{b} m_{b} r_{w}^{2} + J_{b} m_{w} r_{w}^{2} + J_{w} m_{b} r_{b}^{2} + m_{b} m_{w} r_{b}^{2} r_{w}^{2}} - \frac{J_{w} + m_{b} r_{w}^{2} + m_{w} r_{w}^{2}}{J_{b} J_{w} + J_{b} m_{b} r_{w}^{2} + J_{b} m_{w} r_{w}^{2} + J_{w} m_{b} r_{b}^{2} + m_{b} m_{w} r_{b}^{2} r_{w}^{2}}\end{matrix}\right] $$

In [1]:
# Find the numeric values for A and B for the given system
values = {
    r_w: 0.325,
    m_w: 2.5,
    J_w: 0.12675,
    r_b: 0.3,
    m_b: 12.0,
    J_b: 0.8,
    g: 9.81,
    tau: 0.0
}

An = A.subs(values)
Bn = B.subs(values)
display(An)
display(Bn)

NameError: name 'r_w' is not defined

In [102]:
from scipy.signal import place_poles
import numpy as np

# Substitute values
A_num = np.array(An).astype(float)
B_num = np.array(Bn).astype(float)

# Desired pole locations
desired_poles = [-0.7, -0.6, -0.2, -0.5]
# Zeta, Theta, Zeta dot, Theta dot

# Compute gain matrix K
K_place = place_poles(A_num, B_num, desired_poles).gain_matrix

# Compute F_n = A - BK
Fn = A_num - B_num @ K_place

# Compute eigenvalues of F_n
eig_values = np.linalg.eigvals(Fn)

# Output results
print("Computed Gain Matrix K:\n", K_place.tolist())
print("Eigenvalues of F_n:\n", eig_values)

Computed Gain Matrix K:
 [[-0.00639906557933585, -21.593083384086587, -0.06460009061044583, -1.259227839857457]]
Eigenvalues of F_n:
 [-0.7 -0.6 -0.5 -0.2]


In [69]:
print("A:",An.tolist())
print("B:",Bn.tolist())
print("K:",K_place.tolist())

A: [[0, 0, 1, 0], [0, 0, 0, 1], [0, -7.67924619473303, 0, 0], [0, 33.4900459048079, 0, 0]]
B: [[0], [0], [0.566840745627892], [-1.61735461928745]]
K: [[-767.8878695216022, -556.3159542708306, -367.7939121080729, -149.92418637514083]]
